In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the Iris dataset
iris = load_iris()
X = iris.data.T
y = iris.target

# One-hot encode the target labels
num_classes = len(np.unique(y))
y_encoded = np.eye(num_classes)[y]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Define a simple neural network architecture
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Initialize weights and biases
        self.W1 = np.random.randn(hidden_size, input_size)
        self.b1 = np.zeros((hidden_size, 1))
        self.W2 = np.random.randn(output_size, hidden_size)
        self.b2 = np.zeros((output_size, 1))
    
    def forward_propagation(self, X):
        # Calculate hidden layer activations
        Z1 = np.dot(self.W1, X) + self.b1
        A1 = np.maximum(0, Z1)  # ReLU activation
        
        # Calculate output layer logits
        Z2 = np.dot(self.W2, A1) + self.b2
        
        return Z2
    
    def predict(self, X):
        # Perform forward propagation
        Z2 = self.forward_propagation(X)
        
        # Apply softmax to get class probabilities
        exp_scores = np.exp(Z2)
        probabilities = exp_scores / np.sum(exp_scores, axis=0)
        
        # Choose the class with the highest probability as the predicted class
        predictions = np.argmax(probabilities, axis=0)
        return predictions

# Create a neural network instance
input_size = X_train.shape[0]
hidden_size = 10
output_size = num_classes
neural_net = NeuralNetwork(input_size, hidden_size, output_size)

# Train the neural network using forward and backward propagation
def train_neural_network(neural_net, X_train, y_train, num_epochs, learning_rate):
    for epoch in range(num_epochs):
        # Forward propagation
        Z2 = neural_net.forward_propagation(X_train)
        
        # Calculate softmax probabilities
        exp_scores = np.exp(Z2)
        probs = exp_scores / np.sum(exp_scores, axis=0)
        
        # Backpropagation
        error = probs - y_train
        dW2 = np.dot(error, neural_net.forward_propagation(X_train).T) / X_train.shape[1]
        db2 = np.sum(error, axis=1, keepdims=True) / X_train.shape[1]
        
        error_hidden = np.dot(neural_net.W2.T, error) * (neural_net.forward_propagation(X_train) > 0)
        dW1 = np.dot(error_hidden, X_train.T) / X_train.shape[1]
        db1 = np.sum(error_hidden, axis=1, keepdims=True) / X_train.shape[1]
        
        # Update weights and biases
        neural_net.W2 -= learning_rate * dW2
        neural_net.b2 -= learning_rate * db2
        neural_net.W1 -= learning_rate * dW1
        neural_net.b1 -= learning_rate * db1

# Train the neural network
num_epochs = 1000
learning_rate = 0.01
train_neural_network(neural_net, X_train, y_train.T, num_epochs, learning_rate)

# Make predictions using the trained model
predictions = neural_net.predict(X_test)
accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print("Accuracy:", accuracy)


ValueError: Found input variables with inconsistent numbers of samples: [4, 150]